You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [31]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [1]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [33]:

from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [2]:
import os

from langchain_groq import ChatGroq

llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"), model="mixtral-8x7b-32768"
)  # llama3-70b-8192


## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [3]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    llm=llm,
	verbose=True

)

### Agent: Writer

In [4]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

### Agent: Editor

In [5]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    llm = llm,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [6]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [7]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [8]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [9]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [10]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [2024-08-11 21:16:18][DEBUG]: == Working Agent: Content Planner
 [2024-08-11 21:16:18][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now understand the importance and requirements of this task. To create a comprehensive content plan for a blog article about Artificial Intelligence, I will need to conduct thorough research and analysis to prioritize the latest trends, key players, and noteworthy news in the field. Additionally, I will need to identify the target audience, taking into account their interests and pain points, and develop a detailed content outline including an introduction, key points, and a call to action. To ensure the articl

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

The Rise of Artificial Intelligence in Healthcare

Introduction

Artificial Intelligence (AI) is transforming various industries, and healthcare is no exception. The application of AI in healthcare has the potential to revolutionize patient outcomes, streamline workflows, and make healthcare more accessible than ever before. In this article, we explore the latest trends, key players, and noteworthy news in AI in healthcare. We will discuss the benefits and challenges of AI-driven diagnostics, virtual healthcare assistants, and robotic surgery.

Section 1: AI-driven Diagnostics

AI is playing an increasingly critical role in diagnostics, with the potential to detect diseases earlier and more accurately than human clinicians. The benefits of AI-driven diagnostics include early detection, improved accuracy, and reduced costs. However, there are also challenges, such as data privacy and the need for large datasets.

Real-world examples of AI-driven diagnostics include IBM Watson Health's medical imaging solution, which uses AI to analyze medical images and assist radiologists in diagnosing diseases. Google DeepMind's algorithm has also been used to predict acute kidney injury up to 48 hours before it happens, enabling early intervention and improved patient outcomes.

Section 2: Virtual Healthcare Assistants

Virtual healthcare assistants are becoming increasingly popular, with the potential to improve patient engagement and accessibility to healthcare services. The benefits of virtual healthcare assistants include improved patient engagement, increased accessibility, and reduced costs. However, there are also challenges, such as the need for human oversight and potential job displacement.

Examples of current virtual healthcare assistants include Microsoft Healthcare's virtual health assistant, which uses AI to answer patient questions, provide medication reminders, and schedule appointments. Other virtual healthcare assistants include Ada Health, which uses AI to provide personalized health assessments and recommendations, and Babylon Health, which uses AI to provide virtual consultations with healthcare professionals.

Section 3: Robotic Surgery

AI is also playing a significant role in robotic surgery, with the potential to improve precision and minimize invasive procedures. The benefits of robotic surgery include precision, minimally invasive procedures, and reduced recovery time. However, there are also challenges, such as the high cost and the need for specialized training.

Examples of AI-assisted surgeries include the da Vinci Surgical System, which uses AI to assist surgeons in performing minimally invasive procedures. The system has been used in various surgeries, including gynecologic, urologic, and general surgical procedures. Another example is the Medtronic Hugo robotic-assisted surgery system, which uses AI to assist surgeons in performing complex procedures with increased precision and accuracy.

Conclusion

The rise of AI in healthcare has the potential to revolutionize patient outcomes, streamline workflows, and make healthcare more accessible. However, there are also challenges that need to be addressed, such as data privacy, the need for large datasets, job displacement, and the high cost of AI-assisted procedures. As AI continues to evolve and improve, it is essential to stay informed about the latest advancements in healthcare and understand the impact of AI on patient outcomes and workflows.

It is an exciting time for AI in healthcare, and we encourage readers to engage in the conversation and seek more information. By working together, we can ensure that AI is used responsibly and ethically in healthcare, with the potential to improve patient outcomes and make healthcare more accessible for all.

Relevant data and sources:

* IBM Watson Health: <https://www.ibm.com/watson-health/>
* Google DeepMind: <https://deepmind.com/>
* Microsoft Healthcare: <https://www.microsoft.com/en-us/healthcare/>
* FDA: <https://www.fda.gov/>
* Peer-reviewed studies on AI in healthcare: <https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7552827/>
* Industry reports on AI in healthcare: <https://www.mckinsey.com/industries/healthcare-systems-and-services/our-insights/how-ai-could-transform-healthcare>
* News articles on AI in healthcare: <https://www.healthcareitnews.com/news/top-10-ai-stories-2021>

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [12]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [2024-08-11 21:18:37][DEBUG]: == Working Agent: Content Planner
 [2024-08-11 21:18:37][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now understand the importance and requirements of this task. I will gather the necessary information and organize it into a comprehensive content plan for a blog article about the topic: "The Rise of Artificial Intelligence in Healthcare".

Final Answer:

Content Plan: The Rise of Artificial Intelligence in Healthcare

1. Prioritize the latest trends, key players, and noteworthy news:

- Trends: AI-driven diagnostics, virtual healthcare assistants, and robotic surgery.
- Key Players: IBM Watson Health, Google DeepMind,

In [14]:
Markdown(result)

The Rise of Artificial Intelligence in Healthcare

Introduction

Artificial Intelligence (AI) is transforming various industries, and healthcare is no exception. The application of AI in healthcare has the potential to revolutionize patient outcomes, streamline workflows, and make healthcare more accessible than ever before. In this article, we explore the latest trends, key players, and noteworthy news in AI in healthcare. We will discuss the benefits and challenges of AI-driven diagnostics, virtual healthcare assistants, and robotic surgery.

Section 1: AI-driven Diagnostics

AI is playing an increasingly critical role in diagnostics, with the potential to detect diseases earlier and more accurately than human clinicians. The benefits of AI-driven diagnostics include early detection, improved accuracy, and reduced costs. However, there are also challenges, such as data privacy and the need for large datasets.

Real-world examples of AI-driven diagnostics include IBM Watson Health's medical imaging solution, which uses AI to analyze medical images and assist radiologists in diagnosing diseases. Google DeepMind's algorithm has also been used to predict acute kidney injury up to 48 hours before it happens, enabling early intervention and improved patient outcomes.

Section 2: Virtual Healthcare Assistants

Virtual healthcare assistants are becoming increasingly popular, with the potential to improve patient engagement and accessibility to healthcare services. The benefits of virtual healthcare assistants include improved patient engagement, increased accessibility, and reduced costs. However, there are also challenges, such as the need for human oversight and potential job displacement.

Examples of current virtual healthcare assistants include Microsoft Healthcare's virtual health assistant, which uses AI to answer patient questions, provide medication reminders, and schedule appointments. Other virtual healthcare assistants include Ada Health, which uses AI to provide personalized health assessments and recommendations, and Babylon Health, which uses AI to provide virtual consultations with healthcare professionals.

Section 3: Robotic Surgery

AI is also playing a significant role in robotic surgery, with the potential to improve precision and minimize invasive procedures. The benefits of robotic surgery include precision, minimally invasive procedures, and reduced recovery time. However, there are also challenges, such as the high cost and the need for specialized training.

Examples of AI-assisted surgeries include the da Vinci Surgical System, which uses AI to assist surgeons in performing minimally invasive procedures. The system has been used in various surgeries, including gynecologic, urologic, and general surgical procedures. Another example is the Medtronic Hugo robotic-assisted surgery system, which uses AI to assist surgeons in performing complex procedures with increased precision and accuracy.

Conclusion

The rise of AI in healthcare has the potential to revolutionize patient outcomes, streamline workflows, and make healthcare more accessible. However, there are also challenges that need to be addressed, such as data privacy, the need for large datasets, job displacement, and the high cost of AI-assisted procedures. As AI continues to evolve and improve, it is essential to stay informed about the latest advancements in healthcare and understand the impact of AI on patient outcomes and workflows.

It is an exciting time for AI in healthcare, and we encourage readers to engage in the conversation and seek more information. By working together, we can ensure that AI is used responsibly and ethically in healthcare, with the potential to improve patient outcomes and make healthcare more accessible for all.

Relevant data and sources:

* IBM Watson Health: <https://www.ibm.com/watson-health/>
* Google DeepMind: <https://deepmind.com/>
* Microsoft Healthcare: <https://www.microsoft.com/en-us/healthcare/>
* FDA: <https://www.fda.gov/>
* Peer-reviewed studies on AI in healthcare: <https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7552827/>
* Industry reports on AI in healthcare: <https://www.mckinsey.com/industries/healthcare-systems-and-services/our-insights/how-ai-could-transform-healthcare>
* News articles on AI in healthcare: <https://www.healthcareitnews.com/news/top-10-ai-stories-2021>

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).